# 记忆体与多轮对话

1. 传入的不再是一句话，而是一个列表
2. 通过循环的方式来把每次的对话append到列表，使得列表里具有之前的对话,形成“记忆”的作用
3. 在append前，要明确对话信息是 “人类信息” 还是 “AI信息”

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

# 使用 硅基流动 模型
model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="",
)

parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是一名数学智能助手。"),
    MessagesPlaceholder(variable_name="messages"), # 通过占位符MessagePlaceholder定义一个消息列表
])

chain = prompt | model | parser

messages_list = []  # 初始化历史
print("  输入 exit 结束对话")
while True:
    user_query = input("你：")
    if user_query.lower() in {"exit", "quit"}:  # 如果用户的输入存在 exist 则退出
        break

    # 1) 追加用户消息，将user_query转换成 HumanMessage 对象，然后append到消息列表
    messages_list.append(HumanMessage(content=user_query))

    # 2) 调用模型
    assistant_reply = chain.invoke({"messages": messages_list})
    print("助手：", assistant_reply)

    # 3) 追加 AI 回复，将AI回复（assistant_reply）转换成 AIMessage 对象，然后append到消息列表
    messages_list.append(AIMessage(content=assistant_reply))

    # 4) 仅保留最近 50 条
    messages_list = messages_list[-50:]

  输入 exit 结束对话
助手： 

微积分是数学的一个重要分支，主要研究**变化的规律**（微分）和**累积的总量**（积分），是分析连续变化现象的工具。它由两个核心部分组成：**微分学**和**积分学**，两者在数学上是互为逆运算的关系。

---

### **1. 微分学（微分）**
- **核心概念**：研究函数的**瞬时变化率**（导数）。
- **导数的意义**：  
  导数表示函数在某个点的变化速度。例如：  
  - 一个物体运动的**速度**是位移对时间的导数（$\frac{d s}{d t}$）。  
  - 曲线的**切线斜率**（即函数在某一点的导数）。  
  - 经济学中的**边际成本**或**边际收益**（成本或收益的瞬时变化率）。
- **应用**：  
  - 分析函数的极值（最大值、最小值），解决优化问题（如最小化材料用量）。  
  - 描述物理中的加速度、力的变化等动态过程。

---

### **2. 积分学（积分）**
- **核心概念**：研究函数的**累积总量**（积分）。
- **积分的意义**：  
  积分可以计算总量或面积、体积。例如：  
  - 一个变速运动的**总路程**是速度对时间的积分（$\int v(t) dt$）。  
  - 求不规则图形的面积或曲线围成的区域面积。  
  - 在概率中，计算分布函数的累积概率。
- **应用**：  
  - 计算几何中的面积、体积、弧长等。  
  - 解决物理学中的功、电荷等总量问题。  
  - 分析数据的累积趋势，例如求积分可以得到总量。

---

### **3. 微积分的基本定理**
微分和积分是相互关联的：  
- **微分的逆运算**是积分（如导数$\frac{d}{dx}$与积分$\int dx$相反）。  
- **微积分基本定理**指出：一个函数的积分可以通过其导数（原函数）来计算，反之亦然。  
  这是微积分的核心理论基础，连接了微分和积分的桥梁。

---

### **4. 应用领域**
微积分被广泛应用于：  
- **物理**：力学、电磁学、热力学等。  
- **工程**：结构分析、信号处理、控制系统。  
- **经济学**：优化模型、消费与生产函数。  
- **计算机科学**：机器学习算法、计算机图形学、人

# 流式打印

.invoke方法是让模型全部生成完后一次性返回给用户，需要用户等待

流式输出可以做到输出1个token就马上返回给用户，使用 .stream()方法：
1. stream()是同步方法,使用for循环接受返回的输出
2. 如果异步调用，需要使用astream(),然后使用 async for 异步循环获取模型输出

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

# 使用 硅基流动 模型
model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="",
)

parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是一名数学智能助手。"),
    MessagesPlaceholder(variable_name="messages"), # 通过占位符MessagePlaceholder定义一个消息列表
])

chain = prompt | model | parser

messages_list = []  # 初始化历史
print("  输入 exit 结束对话")
while True:
    user_query = input("你：")
    if user_query.lower() in {"exit", "quit"}:  # 如果用户的输入存在 exist 则退出
        break

    # 1) 追加用户消息，将user_query转换成 HumanMessage 对象，然后append到消息列表
    messages_list.append(HumanMessage(content=user_query))

    # 2) 调用模型
    assistant_reply=''
    print('助手：', end=' ')
    for chunk in chain.stream({"messages": messages_list}):
        assistant_reply+=chunk  # 这个是为了存储完整回复内容，用于append到消息列表
        print(chunk, end="", flush=True)
    print() # 换行
    # 3) 追加 AI 回复，将AI回复（assistant_reply）转换成 AIMessage 对象，然后append到消息列表
    messages_list.append(AIMessage(content=assistant_reply))

    # 4) 仅保留最近 50 条
    messages_list = messages_list[-50:]


  输入 exit 结束对话
助手：

要解方程 $x^2 + 10 = 12$，按照以下步骤进行：

1. **整理方程**：将所有项移到等号左边，得到  
   $$
   x^2 + 10 - 12 = 0 \quad \Rightarrow \quad x^2 - 2 = 0
   $$

2. **移项求解**：将常数项移到右边，  
   $$
   x^2 = 2
   $$

3. **开平方**：对两边同时开平方，得到  
   $$
   x = \pm \sqrt{2}
   $$

4. **验证解**：  
   - 当 $x = \sqrt{2}$ 时，代入原方程：  
     $$
     (\sqrt{2})^2 + 10 = 2 + 10 = 12
     $$  
   - 当 $x = -\sqrt{2}$ 时，代入原方程：  
     $$
     (-\sqrt{2})^2 + 10 = 2 + 10 = 12
     $$  
   两者均满足方程。

**最终解**：  
$$
x = \sqrt{2} \quad \text{或} \quad x = -\sqrt{2}
$$

答案：$x = \pm \sqrt{2}$


# LangChain接入自定义工具

In [ ]:
import requests
from langchain_core.tools import tool

@tool
def get_weather(loc):
    """
        查询即时天气函数
        :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
        :return：心知天气 API查询即时天气的结果，具体URL请求地址为："https://api.seniverse.com/v3/weather/now.json"
        返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    url = "https://api.seniverse.com/v3/weather/now.json"
    params = {
        "key": "你注册的心知天气api key",
        "location": loc,
        "language": "zh-Hans",
        "unit": "c",
    }
    response = requests.get(url, params=params)
    temperature = response.json()
    return temperature['results'][0]['now']

print(get_weather.name)
print(get_weather.description)
print(get_weather.args)

In [ ]:
from langchain.chat_models import init_chat_model

# 使用 硅基流动 模型
model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="",
)

tools = [get_weather]
llm_with_tools = model.bind_tools(tools)  # 将工具绑定到模型上
response = llm_with_tools.invoke("你好， 请问北京的天气怎么样？")

print(response)

In [ ]:
parser = JsonOutputKeyToolsParser(key_name=get_weather.name, first_tool_only=True)

llm_chain = llm_with_tools | parser

response = llm_chain.invoke("你好， 请问北京的天气怎么样？")

print(response)

In [ ]:
get_weather_chain = llm_chain | get_weather
response = get_weather_chain.invoke("请问上海今天天气如何?")
print(response)